## Elements of a Prompt

In [7]:
import os
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

In [8]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

chat_model = ChatOpenAI()

conversation_chain = ConversationChain(
    llm = chat_model
)


print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


### Breaking down the elements of the prompt

Here we will analyze the different elements of the prompt.

`The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:`

In the above prompt, 
`The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.` - is the instructions for LLM, what the model will do.

`Current conversation:, Human: {input}` are context.

`{history}` is the additional context.

`{input}` - Input data from human for LLM.

`AI:` - Output Indicator.

## FEW SHOT LEARNING

A way to provide some additional context to the LLM in a form of examples, such that it understands how to return the output.

{Instructions}

{Context}:{Examples} <- Additional Context

{Context}:{User Input}

{Output Indicator}:

In [9]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

custom_examples = [
    {'input': '2+2', 'output': '4'},
    {'input': '2+3', 'output': '5'},
    {'input': '5+6', 'output': '11'},
]

# we use from_message rather than from chatprompt template
custom_example_prompt = ChatPromptTemplate.from_messages(
    [
        ('human', '{input}'),
        ('ai', '{output}')
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = custom_example_prompt,
    examples = custom_examples,
)

# checking the format of the prompt
print(few_shot_prompt.format())



Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11


In [10]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are wonderous wizard of math.'),
        few_shot_prompt,
        ('human','{input}')
    ]
)

print(final_prompt.format(input='What is 3+4'))

System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+4


In [11]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm = chat_model,
    prompt = final_prompt,
    verbose = True
)

chain.invoke('What is 3+4')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+4

> Finished chain.


{'input': 'What is 3+4', 'text': '3 + 4 = 7'}

The more we give examples to our few short prompt, the more accurate our llm model output will be.


## Memetic Proxy

Is an interesting technique where we provide a cultural reference or analogy to the LLM such that he understands a bit better what he needs to do.


In [12]:
from langchain.prompts import PromptTemplate

template = '''
System: {reference}
Provide an helpful answer to the following question

Human: {question}

AI:
'''

prompt = PromptTemplate.from_template(template)

chain = LLMChain(
    llm = chat_model,
    prompt = prompt,
    verbose = True
)

high_level = 'Imagine you are a Professor teaching at the PhD level.'

low_level = 'Imagine you are a Kindergarten techer.'

question = 'What is Quantum Mechanics'

chain.run({
    'question': question,
    'reference': high_level
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Professor teaching at the PhD level.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:


> Finished chain.


'Quantum Mechanics is a branch of physics that deals with the behavior of particles at the smallest scales, such as atoms and subatomic particles. It involves principles such as quantization, superposition, and entanglement, which can be quite different from classical physics. Quantum Mechanics is essential for understanding the behavior of matter and energy at the microscopic level and has led to significant technological advancements in areas such as computing, communication, and materials science.'

In [13]:
chain.run({
    'question': question,
    'reference': low_level
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Kindergarten techer.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:


> Finished chain.


'As a Kindergarten teacher, you can explain Quantum Mechanics in a simple way by saying, "Quantum Mechanics is a science that helps us understand how really tiny things, like atoms and particles, work. It\'s like a special puzzle that scientists use to learn more about the world around us."'

## CHAIN OF THOUGHT

**Standard Prompting**

Here we ask a question to LLM and give a sample answer for the question. Then again, we ask another question and LLM provide us with an answer.

**Chain of Thought Prompting**

Here, with addition to the answer, we provide a rationale to the answer. This helps the LLM to think through the answer. **Chain of thoughts** require an example.

Dataset is from [Kaggle](https://www.kaggle.com/datasets/konradb/chain-of-thought-collection/data?select=CoT_collection.json)

In [14]:
import pandas as pd

cot_df = pd.read_json('CoT_collection.json')
 
cot_df.head(5)

,407496,897822,1639180,1268206,112965,176714,1365585,1266292,306879,765453,...,1155268,891128,558286,1432572,471785,69445,1810512,1472056,570474,359628
source,Article: Phytochemistry is a branch of plant b...,"In this task, you are provided with an article...",What's the answer to that question: where did ...,Extract the answer to the question from the fo...,"Which entity is this text about?\n\nRichard ""R...",The town lies along the stretch of coastline d...,Two sisters went to the hospital. The medics r...,"Question: ""What else does the pigment make red...","""Mama came over and sat down beside Alice. Gen...","Given a travel related situation, and given a ...",...,Read the given text and if it has abusive cont...,"In this task, you will be shown a prompt from ...","In this task, you're given the title of a five...",Read the following context and answer the ques...,You will be given a passage consisting of set ...,Answer the following question:\n\nthe person w...,Exercise: read the text and answer the questio...,Just minutes after Donald Trump fired Rex Till...,"In this task, you're given four sentences of a...",Read this article and answer this question The...
target,from plants,Regulation,Battle of Appomattox Court House,major heart attacks,Red Skelton,no,Sue,lips,no,Where to watch German world cup soccer games i...,...,no,holding this court may affirm on any grounds s...,a,The narrator feels insecure about her own faith .,kitchen,designer,True,A,A,4
rationale,The article states that many medicinal and rec...,This act has all the features of Regulation. T...,The Battle of Appomattox Court House was a bat...,The answer is the number of major heart attack...,The text is about Red Skelton. The context des...,The paragraph doesn't mention the date of the ...,Sickle cell trait is a hereditary disorder in ...,The context mentions that the pigment pheomela...,The premise states that Mama came over and sat...,The question asks for a location to watch Germ...,...,"The given text does not contain any abuse, neg...","The first option is ""holding this court may af...",Sentence a is not connected with the rest of t...,The narrator mentions that she grew up in a no...,We can infer from the last fact in the passage...,"The answer is ""designer""\nA database designer ...",The text explains that bumpers are legally req...,The question of what Tillerson said Trump shou...,"Sentence4 shows that the man is ill and sad, s...","To solve the problem, we need to find a month ..."
config,none,task_744,none,dbert,none,none,none,dbert,none,task_1639,...,task_108,task_287,task_214,none,task_083,none,boolq,none,task_296,none
task,squad_v1,eurlex,web_questions,adversarial_qa@dbert,quac,rte,ropes,adversarial_qa@dbert,wnli,diqa,...,cad,casehold,roc_stories,cosmos_qa,babi,natural_questions,super_glue@boolq,quail,roc_stories,drop


In [15]:
cot_df = cot_df.T
cot_df.head(5)

,source,target,rationale,config,task,prompt
407496,Article: Phytochemistry is a branch of plant b...,from plants,The article states that many medicinal and rec...,none,squad_v1,6
897822,"In this task, you are provided with an article...",Regulation,This act has all the features of Regulation. T...,task_744,eurlex,none
1639180,What's the answer to that question: where did ...,Battle of Appomattox Court House,The Battle of Appomattox Court House was a bat...,none,web_questions,whats_the_answer
1268206,Extract the answer to the question from the fo...,major heart attacks,The answer is the number of major heart attack...,dbert,adversarial_qa@dbert,based_on
112965,"Which entity is this text about?\n\nRichard ""R...",Red Skelton,The text is about Red Skelton. The context des...,none,quac,8


In [16]:
len(cot_df)

1837928

In [17]:
cot_df = cot_df[cot_df.task == 'ai2_arithmetic_questions']
len(cot_df)

580

In [18]:
cot_df.reset_index(inplace=True, drop=True)
cot_df

,source,target,rationale,config,task,prompt
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...,task_745,ai2_arithmetic_questions,none
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5,task_745,ai2_arithmetic_questions,none
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...,task_745,ai2_arithmetic_questions,none
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...,task_745,ai2_arithmetic_questions,none
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...,task_745,ai2_arithmetic_questions,none
...,...,...,...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",task_745,ai2_arithmetic_questions,none
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,task_745,ai2_arithmetic_questions,none
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...,task_745,ai2_arithmetic_questions,none
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...,task_745,ai2_arithmetic_questions,none


In [19]:
cot_df = cot_df[['source', 'target', 'rationale']]
cot_df

,source,target,rationale
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...
...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s..."
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...


In [20]:
cot_df.rename(
    columns = {
        'source': 'Question',
        'target': 'Response',
        'rationale': 'Rationale'
    }, 
    inplace=True)
cot_df

/var/folders/__/02h4kb7x4d94q5008bzqg4gm0000gn/T/ipykernel_78666/1618394725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cot_df.rename(


,Question,Response,Rationale
0,You are provided with an arithmetic question. ...,0.6,Since Blake filled the bucket with 0.8 gallon ...
1,You are provided with an arithmetic question. ...,0.5,0.2 + 0.3 = 0.5
2,You are provided with an arithmetic question. ...,0.9,Craig walked 0.2 mile from school to David's h...
3,You are provided with an arithmetic question. ...,0.9,Hannah's Vegetarian Restaurant bought 0.3 poun...
4,You are provided with an arithmetic question. ...,12.3,Alyssa spent $5.71 on football and $6.59 on ma...
...,...,...,...
575,You are provided with an arithmetic question. ...,0.3,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s..."
576,You are provided with an arithmetic question. ...,2.0,The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...
577,You are provided with an arithmetic question. ...,34.7,Mary loves eating fruits. Mary paid $11.08 for...
578,You are provided with an arithmetic question. ...,81.0,Joan has 40 blue balloons Melanie has 41 blue ...


In [21]:
# converting the dataframe to dict to list
list(cot_df.T.to_dict().values())

[{'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nBlake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?",
  'Response': '0.6',
  'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon'},
 {'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nThere is 0.2 cup of oil in Scarlett's measuring cup. If Scarlett adds 0.3 cup more, how m

In [22]:
list(cot_df.T.to_dict().values())[0]

{'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nBlake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?",
 'Response': '0.6',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon'}

In [23]:
import random

random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nAlec and his roommates ate 3.25 pints of ice cream on Friday nightand 0.25 pint of ice cream on Saturday night. How many pints didthey eat in all?",
  'Response': '3.5',
  'Rationale': 'Alec and his roommates ate 3.25 pints of ice cream on Friday nightand 0.25 pint of ice cream on Saturday night. The number of pints atein all is 3.25 + 0.25 = 3.50'},
 {'Question': "You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.\n\nMike found 6 seashells and 4 starfish, but 4 of the seashells wereb

In [24]:
from langchain.prompts import (
    FewShotPromptTemplate,
    PromptTemplate
)

custom_examples = random.sample(list(cot_df.T.to_dict().values()), 10)

custom_template = '''
Question: {Question}
Rationale: {Rationale}
Response: {Response}
'''

custom_example_prompt = PromptTemplate(
    input_variables = ['Question', 'Rationale', 'Response'],
    template = custom_template,
)

system_prompt = """You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place."""

cot_prompt = FewShotPromptTemplate(
    prefix = system_prompt,
    example_prompt = custom_example_prompt,
    examples = custom_examples,
    suffix = 'Question: {input}', # User input
    input_variables =['input']
)

print(cot_prompt.format(input='[EXAMPLE QUESTION]'))

You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 + 139.38 + 112.46 = 387.9
  ""
Response: 387.9



Question: You are provided wi

In [25]:
chain = LLMChain(
    llm = chat_model,
    prompt = cot_prompt,
    verbose = True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 + 139.38 + 112.46 =

'Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg in her apple pie. To find out how much more cinnamon than nutmeg Mandy used, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1 tablespoon\nResponse: 0.1'

## Self-Consistency

When we run prompts with LLM, we often get different responses, Due to the parameter within the LLM that we can adjust to get more or less deterministic responses.

Self-consistency is about using the same question and running different chain of thoughts to see the different responses and the way to mitigate the variation we get about the different answer is to take the answe that appears more often, in other words take the answer which is more consistent accross different chain of thoughts.


In [26]:
# if we want deterministic responses, we need to set the `temperature` to 0 
chat_model = ChatOpenAI(temperature=0.7) # responses are not deterministic and different every time

custom_cot_chain = LLMChain(
    llm = chat_model,
    prompt = cot_prompt,
    verbose = True
)


template = '''Using the following answers, 
return the answer that comes up more often 

{answers}


ANSWER:'''


consistency_prompt = PromptTemplate.from_template(template)

consistency_chain = LLMChain(
    llm = chat_model,
    prompt = consistency_prompt,
    verbose = True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

responses = []

for i in range(10):
    response = custom_cot_chain.run(question)
    responses.append(response)



> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 + 139.38 + 112.46 =


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Question: You are provided with an arithmetic question. Your task is to compute the solution using the given arithmetic operations. The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.

Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He wanted 3 CD's for $6.16, but did n't buythem. In total, how much did he spend?
Rationale: Keith spent $136.06 on speakers, $139.38 on a CD player, and $112.46 on new tires. He didn't buy the CDs, so those can be ignored in the answer (Total cost of Keith's purchase) = 136.06 


> Finished chain.


In [27]:
answers = '\n\n'.join(responses)
consistency_chain.run(answers)



> Entering new LLMChain chain...
Prompt after formatting:
Using the following answers, 
return the answer that comes up more often 

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To find out how much more cinnamon than nutmeg Mandy used, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1 tablespoons
Response: 0.1

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To find out how much more cinnamon than nutmeg Mandy used, subtract the amount of nutmeg from the amount of cinnamon, which is 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. To find out how much more cinnamon than nutmeg Mandy used, subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. To find out how much more cinnamon than nutmeg Mandy used, we

'0.1'

## Inception

Inception is known as the zero-shot Chain of Thought. Instead of having examples where we showcase the rationale for how to solve a problem, model is being asked to think step by step, this leads the model to provide a set of steps that needs to happen that solves the problem.

In [28]:
system_prompt = """You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place."""


template = '''

{input}

Let's think about it step by step:
'''

prompt = PromptTemplate.from_template(system_prompt + template)

chain = LLMChain(
    llm = chat_model,
    prompt = prompt,
    verbose = True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""


chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:
You are provided with an arithmetic question. Your task is to compute the solution using 
the given arithmetic operations. The only arithmetic operators needed to answer the questions 
are'+'(addition) and'-'(subtraction). The answer should be correct to one decimal place.


Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?


Let's think about it step by step:


> Finished chain.


'Step 1: Calculate how much more cinnamon than nutmeg Mandy used.\n0.6 tablespoons (cinnamon) - 0.5 tablespoons (nutmeg) = 0.1 tablespoons\n\nTherefore, Mandy used 0.1 tablespoon more cinnamon than nutmeg.'

## Self-Ask

Self-ask is a similar technique to Chain of Thought, but instead, we request intermediate questions and answers for the LLM to follow a logical path to get the answer.

### Chain of Thought Prompting
**Model Input:** Question -> Answer -> Question

**Model Answer:** Answer

### Self-Ask Prompting
**Model Input:** Question -> Follow up Questions & Answers (Until it's confident) -> Question

**Model Answer:** Answer

Self-Ask prompting technique is very well adapted to interact with search engines. The intermediate question that can be sent to a search engine and the search engine could provide the answer. Based on the answer we can have additional intermediate question and get another intermediate answer. Then, we can have the final answer based on the interaction we have with the search engine and the LLM.

In [32]:
# installing necessary packages
# %pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=fe1771fb576f464ceaa5403b2080a0bf577360309d36e35f237f3a570e21a6fa
  Stored in directory: /Users/shuvobarman/Library/Caches/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [30]:
os.environ['SERPAPI_API_KEY']='SERP_API_KEY'

In [31]:
from langchain.agents import(
    initialize_agent,
    AgentType,
    load_tools
)

tools = load_tools(['serpapi'])

# renaming the tool name
tools[0].name = 'Intermediate Answer'

# initialize the agents
self_ask_with_search = initialize_agent(
    tools = tools,
    llm = chat_model,
    agent = AgentType.SELF_ASK_WITH_SEARCH,
    verbose = True
)

# look at the prompt template
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

/Users/shuvobarman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [32]:
# importing libraries to print all the followup questions
from langchain.callbacks import StdOutCallbackHandler

# instantiale the callback handler
handler = StdOutCallbackHandler()

question = 'What is the hometown of 2019 F1 World Champion?'

self_ask_with_search.run(
    question,
    callbacks=[handler]
    
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mot


> Finished chain.
So the final answer is: Stevenage

> Finished chain.


'Stevenage'

## ReAct

ReAct (Resonning and Acting) is a technique to push the LLM to make an action after thinking about it. This is very well adapted to augment LLMs with tools.

**Workflow:**

Question -> Reasoning -> Action -> Answer

In [33]:
tools = load_tools(['serpapi', 'llm-math'], llm=chat_model)
agent = initialize_agent(
    tools = tools,
    llm = chat_model,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

handler = StdOutCallbackHandler()

question = '''
Who has the most fastest lap in F1 history?
What was the speed of the car in that lap raised to the 0.43 power?
'''

agent.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
Who has the most fastest lap in F1 history?
What was the speed of the car in that lap raised to the 0.43 power?

Thought:

> Finished chain.
I need to find out who 

"The speed of the car in Hamilton's 2020 pole lap raised to the 0.43 power is approximately 10.99"

In [34]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## Plan and Execute

ReAct (Resonning and Acting) is a technique to push the LLM to make an action after thinking about it. This is very well adapted to augment LLMs with tools.

The idea is to send a question to the LLM and to ask the LLM to establish a set of steps to solve that problem. Each set can be sent to react chain and the result of that react chain can be sent to the next step. We can iterate through this process with each of the steps having its own react chain and at the end we can get the final answer.


In [35]:
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner
)

tools = load_tools(['serpapi', 'llm-math'], llm=chat_model)

planner = load_chat_planner(chat_model)

executor = load_agent_executor(
    chat_model,
    tools,
    verbose = True
)

agent = PlanAndExecute(
    planner = planner,
    executor = executor,
    verbose = True
)

In [36]:
planner.llm_chain.prompt.format(input='Some input')

"System: Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'\nHuman: Some input"

In [37]:
print(executor.chain.agent.llm_chain.prompt.format(
    previous_steps = 'previous_steps', # fake values
    current_step = 'curret_step', # fake value
    agent_scratchpad = 'agent_scratchpad' # fake value
))

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {'tool_input': {'type': 'string'}}
Calculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

In [40]:
question = '''
Who is the current president of United States?
What is the current president age raise to the 0.43 power?
'''


agent.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value='Look up the current president of the United States.'), Step(value='Find the age of the current president.'), Step(value="Calculate the current president's age raised to the power of 0.43."), Step(value='Return the calculated value.'), Step(value="Given the above steps taken, please respond to the user's original question.\n")]

> Entering new AgentExecutor chain...
Thought: The user's current objective is to look up the current president of the United States. Since this information has already been provided as Joe Biden, I can simply confirm this to the user.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Joe Biden"
}
``` 


> Finished chain.
*****

Step: Look up the current president of the United States.

Response: Joe Biden

> Entering new AgentExecutor chain...
Thought: The age of the current president has already been determined in a previous step (81 years). I can simply provide this information to the user

"The driver who holds the record for the fastest lap in Formula 1 history is Valtteri Bottas. The current president of the United States is Joe Biden, who is 81 years old. The calculated value of the current president's age raised to the power of 0.43 is 6.62."